In [1]:
import pickle as pkl
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import APPNPConv
from sklearn.metrics import roc_auc_score
import numpy as np
import torch.backends.cudnn as cudnn
import random
import pickle as pkl

import dgl
import torch
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from sklearn.metrics import roc_auc_score
import numpy as np
import torch.backends.cudnn as cudnn
import random
import pickle as pkl
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from dgl.nn import GraphConv

In [2]:
import dgl
import torch
import torch.nn as nn
from dgl.nn import GatedGraphConv

class GGSNNModel(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes, n_steps, n_etypes):
        super(GGSNNModel, self).__init__()
        self.ggsnn_layers = nn.ModuleList()
        self.ggsnn_layers.append(GatedGraphConv(in_dim, hidden_dim, n_steps, n_etypes))
        self.ggsnn_layers.append(GatedGraphConv(hidden_dim, hidden_dim, n_steps, n_etypes))
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, g, features):
        h = features
        for layer in self.ggsnn_layers:
            h = layer(g, h)
        h = self.fc(h)
        return h


In [3]:
# load /home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_test_edge_indices.pkl, /home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_train_edge_indices.pkl, /home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_validation_edge_indices.pkl, /home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_test_edge_indices.pkl, /home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_train_edge_indices.pkl, /home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_validation_edge_indices.pkl
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_test_edge_indices.pkl', 'rb') as f:
    positive_test_edge_indices = pkl.load(f)
    
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_train_edge_indices.pkl', 'rb') as f:
    positive_train_edge_indices = pkl.load(f)
    
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/positive_validation_edge_indices.pkl', 'rb') as f:
    positive_validation_edge_indices = pkl.load(f)
    
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_test_edge_indices.pkl', 'rb') as f:
    negative_test_edge_indices = pkl.load(f)
    
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_train_edge_indices.pkl', 'rb') as f:
    negative_train_edge_indices = pkl.load(f)
    
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/negative_validation_edge_indices.pkl', 'rb') as f:
    negative_validation_edge_indices = pkl.load(f)

In [4]:
hidden_dim = 128
num_classes = 128
n_steps = 5
n_etypes = 1

In [5]:
# load G_dgl_training
import dgl
with open('/home/qian/HNE/Model/GCN/Ethereum/matching_link/G_dgl_training', 'rb') as f:
    G_dgl_training = pkl.load(f)

In [6]:
# define generate_edge_embeddings function
def generate_edge_embeddings(h, edges):
    # Extract the source and target node indices from the edges
    src, dst = edges[0], edges[1]
    
    # Use the node indices to get the corresponding node embeddings
    src_embed = h[src]
    dst_embed = h[dst]

    # Concatenate the source and target node embeddings
    edge_embs = torch.cat([src_embed, dst_embed], dim=1)

    return edge_embs

In [7]:
# write a loop to run 5 times of the model and get the average performance
import copy
for i in range(5):
    model = GGSNNModel(16, hidden_dim, num_classes, n_steps, n_etypes)
    # model = model.to('cuda:1')
    # train on positive edges, negative edges; also use validation edges to stop epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    criterion = nn.BCEWithLogitsLoss()

    best_val_loss = float('inf')
    best_model = None
    num_epochs = 200
    patience = 10
    early_stopping_counter = 0
    
    
    transform = nn.Sequential(
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1))
    
    for epoch in range(num_epochs):
        model.train()
        
        # forward pass
        logits = model(G_dgl_training, G_dgl_training.ndata['features'].float())
        
        # generate edge embeddings
        pos_train_edge_embs = generate_edge_embeddings(logits, positive_train_edge_indices)
        neg_train_edge_embs = generate_edge_embeddings(logits, negative_train_edge_indices)
        
        # concatenete positive and negative edge embeddings
        train_edge_embs = torch.cat([pos_train_edge_embs, neg_train_edge_embs], dim=0)
        train_edge_labels = torch.cat([torch.ones(pos_train_edge_embs.shape[0]), torch.zeros(neg_train_edge_embs.shape[0])], dim=0).unsqueeze(1)
        
        # print shapes of tensors for debugging
        # print(f"Train Edge Embeddings Shape: {train_edge_embs.shape}")
        # print(f"Train Edge Labels Shape: {train_edge_labels.shape}")
        
        # calculate loss
        loss = criterion(transform(train_edge_embs), train_edge_labels)
        print(f"Training Loss: {loss.item()}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        # validation
        model.eval()
        
        with torch.no_grad():
            # repeat the same process as above for validation samples
            logits = model(G_dgl_training, G_dgl_training.ndata['features'].float())
            pos_val_edge_embs = generate_edge_embeddings(logits, positive_validation_edge_indices)
            neg_val_edge_embs = generate_edge_embeddings(logits, negative_validation_edge_indices)
            val_edge_embs = torch.cat([pos_val_edge_embs, neg_val_edge_embs], dim=0)
            val_edge_labels = torch.cat([torch.ones(pos_val_edge_embs.shape[0]), torch.zeros(neg_val_edge_embs.shape[0])], dim=0).unsqueeze(1)
            # # print shapes of tensors for debugging
            # print(f"Validation Edge Embeddings Shape: {val_edge_embs.shape}")
            # print(f"Validation Edge Labels Shape: {val_edge_labels.shape}")

            val_loss = criterion(transform(val_edge_embs), val_edge_labels)
            print(f"Validation Loss: {val_loss.item()}")
            
            # early stopping based on validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                # add patience
                early_stopping_counter = 0
                # # save the best model
                best_model = copy.deepcopy(model)
            
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print("Early Stopping!")
                    break
                
    # switch to evaluation mode
    best_model.eval()

    with torch.no_grad():
        # generate the embeddings using the best model
        logits = best_model(G_dgl_training, G_dgl_training.ndata['features'].float())

        # generate edge embeddings for the test samples
        pos_test_edge_embs = generate_edge_embeddings(logits, positive_test_edge_indices)
        neg_test_edge_embs = generate_edge_embeddings(logits, negative_test_edge_indices)

        # concatenate the positive and negative edge embeddings and labels
        test_edge_embs = torch.cat([pos_test_edge_embs, neg_test_edge_embs], dim=0)
        test_edge_labels = torch.cat([torch.ones(pos_test_edge_embs.shape[0]), torch.zeros(neg_test_edge_embs.shape[0])], dim=0)


        # test_loss = criterion(linear(test_edge_embs), val_edge_labels)
        # calculate predictions using the linear layer
        
        predictions = torch.sigmoid(transform(test_edge_embs))
        
        # reshape the predictions and the labels
        predictions = predictions.view(-1).cpu().numpy()
        test_edge_labels = test_edge_labels.cpu().numpy()

        # calculate scores and entropyloss
        
        
        auc = roc_auc_score(test_edge_labels, predictions)
        # here use 0.5 as threshold
        predictions_binary = (predictions > 0.5).astype(int)
        f1 = f1_score(test_edge_labels, predictions_binary)
        precision = precision_score(test_edge_labels, predictions_binary)
        recall = recall_score(test_edge_labels, predictions_binary)
        accuracy = accuracy_score(test_edge_labels, predictions_binary)
    # also record loss
    # print(f"Test Loss: {criterion(transform(test_edge_embs), test_edge_labels)}")
    print(f"AUC: {auc}")
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")
    
    # write the result to a txt file
    with open('result.txt', 'a') as f:
        # write auc, f1, precision, recall
        f.write(f"AUC: {auc}, F1 Score: {f1}, Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}\n")

/home/qian/miniconda3/envs/pytorch/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Training Loss: 0.6974669694900513
Validation Loss: 0.6798465847969055
Training Loss: 0.6816629767417908
Validation Loss: 0.667063295841217
Training Loss: 0.6704307198524475
Validation Loss: 0.6573352217674255
Training Loss: 0.6612285375595093
Validation Loss: 0.6514893770217896
Training Loss: 0.6548253893852234
Validation Loss: 0.6502668857574463
Training Loss: 0.65345299243927
Validation Loss: 0.6401025056838989
Training Loss: 0.6453787088394165
Validation Loss: 0.6355783343315125
Training Loss: 0.641393780708313
Validation Loss: 0.6281944513320923
Training Loss: 0.634535014629364
Validation Loss: 0.6223641037940979
Training Loss: 0.6285625696182251
Validation Loss: 0.6157921552658081
Training Loss: 0.622075080871582
Validation Loss: 0.6096449494361877
Training Loss: 0.6161215305328369
Validation Loss: 0.603866457939148
Training Loss: 0.6104531288146973
Validation Loss: 0.5990932583808899
Training Loss: 0.6057547330856323
Validation Loss: 0.5934596657752991
Training Loss: 0.5995878577

/home/qian/miniconda3/envs/pytorch/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Training Loss: 0.6903670430183411
Validation Loss: 0.6894035935401917
Training Loss: 0.6880025863647461
Validation Loss: 0.6858927607536316
Training Loss: 0.6829702854156494
Validation Loss: 0.6824223399162292
Training Loss: 0.6795192360877991
Validation Loss: 0.6705629825592041
Training Loss: 0.6720179319381714
Validation Loss: 0.663253664970398
Training Loss: 0.6621507406234741
Validation Loss: 0.6572253704071045
Training Loss: 0.6571654081344604
Validation Loss: 0.6551492810249329
Training Loss: 0.655786395072937
Validation Loss: 0.6446884870529175
Training Loss: 0.6455451250076294
Validation Loss: 0.6413326263427734
Training Loss: 0.6391246318817139
Validation Loss: 0.6361405253410339
Training Loss: 0.6333199739456177
Validation Loss: 0.6312155723571777
Training Loss: 0.628402054309845
Validation Loss: 0.62555330991745
Training Loss: 0.6214650869369507
Validation Loss: 0.618834912776947
Training Loss: 0.6153052449226379
Validation Loss: 0.614136815071106
Training Loss: 0.6104447841

/home/qian/miniconda3/envs/pytorch/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Training Loss: 0.6858461499214172
Validation Loss: 0.66910320520401
Training Loss: 0.6721528768539429
Validation Loss: 0.657077431678772
Training Loss: 0.6613848805427551
Validation Loss: 0.6443535685539246
Training Loss: 0.6481220722198486
Validation Loss: 0.6303518414497375
Training Loss: 0.6338472366333008
Validation Loss: 0.6166419386863708
Training Loss: 0.6209592223167419
Validation Loss: 0.6080622673034668
Training Loss: 0.6110637784004211
Validation Loss: 0.6021093130111694
Training Loss: 0.6040791869163513
Validation Loss: 0.5960959792137146
Training Loss: 0.5964995622634888
Validation Loss: 0.5893325805664062
Training Loss: 0.5892753005027771
Validation Loss: 0.5902849435806274
Training Loss: 0.5854856967926025
Validation Loss: 0.5849979519844055
Training Loss: 0.5843930840492249
Validation Loss: 0.5801673531532288
Training Loss: 0.5777051448822021
Validation Loss: 0.5745137333869934
Training Loss: 0.5693227052688599
Validation Loss: 0.5719087719917297
Training Loss: 0.562688

/home/qian/miniconda3/envs/pytorch/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Training Loss: 0.7091658711433411
Validation Loss: 0.692808210849762
Training Loss: 0.6933077573776245
Validation Loss: 0.6887339353561401
Training Loss: 0.6888002753257751
Validation Loss: 0.6819738149642944
Training Loss: 0.6847784519195557
Validation Loss: 0.6783925890922546
Training Loss: 0.6806120872497559
Validation Loss: 0.6715900897979736
Training Loss: 0.6744436621665955
Validation Loss: 0.6625306010246277
Training Loss: 0.6659829020500183
Validation Loss: 0.6589308977127075
Training Loss: 0.6595575213432312
Validation Loss: 0.6462376713752747
Training Loss: 0.650007963180542
Validation Loss: 0.6356269717216492
Training Loss: 0.639965832233429
Validation Loss: 0.6267139315605164
Training Loss: 0.6316736936569214
Validation Loss: 0.6168758273124695
Training Loss: 0.6230121850967407
Validation Loss: 0.6077420115470886
Training Loss: 0.6146805882453918
Validation Loss: 0.5992709994316101
Training Loss: 0.6074211001396179
Validation Loss: 0.5911045670509338
Training Loss: 0.599802

/home/qian/miniconda3/envs/pytorch/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Training Loss: 0.7012495398521423
Validation Loss: 0.6843201518058777
Training Loss: 0.6859338879585266
Validation Loss: 0.6764212250709534
Training Loss: 0.6778358817100525
Validation Loss: 0.667879045009613
Training Loss: 0.668939471244812
Validation Loss: 0.6605783700942993
Training Loss: 0.6613479852676392
Validation Loss: 0.6531836986541748
Training Loss: 0.6533030271530151
Validation Loss: 0.6487224102020264
Training Loss: 0.6508572101593018
Validation Loss: 0.6417062282562256
Training Loss: 0.6447780132293701
Validation Loss: 0.6354227066040039
Training Loss: 0.6369538903236389
Validation Loss: 0.6301252841949463
Training Loss: 0.6301214694976807
Validation Loss: 0.6264376640319824
Training Loss: 0.6242250800132751
Validation Loss: 0.6226979494094849
Training Loss: 0.6165881156921387
Validation Loss: 0.6185597777366638
Training Loss: 0.6108192801475525
Validation Loss: 0.6154117584228516
Training Loss: 0.6042417287826538
Validation Loss: 0.6175864338874817
Training Loss: 0.59710